### Полезные ссылки:

https://developers.google.com/google-ads/api/docs/campaigns/overview

https://developers.google.com/google-ads/api/docs/client-libs/python

https://pypi.org/project/google-ads/

https://github.com/googleads/google-ads-python/#getting-started

Приложение:

https://console.cloud.google.com/apis/dashboard?hl=ru&project=ace-axon-365313

In [1]:
# pip install google-ads

In [2]:
import pandas as pd
import numpy as np

import argparse
import sys
from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException

In [3]:
client = GoogleAdsClient.load_from_storage("google-ads-intapichecking@gmail.com.yaml")

In [4]:
# получение кампаний
# https://developers.google.com/google-ads/api/docs/first-call/get-campaigns#python

def get_campaigns(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")

    query = """
        SELECT
          campaign.id,
          campaign.name
        FROM campaign
        ORDER BY campaign.id"""

    # Issues a search request using streaming.
    stream = ga_service.search_stream(customer_id=customer_id, query=query)
    result = []
    for batch in stream:
#         print(type(batch))
        for row in batch.results:
            result.append({'campaign_id': row.campaign.id,
                             'campaign_name': row.campaign.name})
#             print(
#                 f"Campaign with ID {row.campaign.id} and name "
#                 f'"{row.campaign.name}" was found.'
#             )
    return result

In [5]:
campaigns = get_campaigns(client, customer_id='2200886572')

In [6]:
pd.DataFrame(campaigns)

,campaign_id,campaign_name
0,18616867140,Sales-Search-1


In [15]:
# общий вариант использования отчета
# https://developers.google.com/google-ads/api/docs/reporting/example

def basic_report(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")

    query = """
        SELECT
          campaign.id,
          campaign.name,
          ad_group.id,
          ad_group.name,
          ad_group_criterion.criterion_id,
          ad_group_criterion.keyword.text,
          ad_group_criterion.keyword.match_type,
          metrics.impressions,
          metrics.clicks,
          metrics.cost_micros
        FROM keyword_view WHERE segments.date DURING LAST_7_DAYS
        AND campaign.advertising_channel_type = 'SEARCH'
        AND ad_group.status = 'ENABLED'
        AND ad_group_criterion.status IN ('ENABLED', 'PAUSED')
        ORDER BY metrics.impressions DESC
        LIMIT 50
        """

    # Issues a search request using streaming.
    search_request = client.get_type("SearchGoogleAdsStreamRequest")
    search_request.customer_id = customer_id
    search_request.query = query
    stream = ga_service.search_stream(search_request)
    result = []
    for batch in stream:
        for row in batch.results:
            campaign = row.campaign
            ad_group = row.ad_group
            criterion = row.ad_group_criterion
            metrics = row.metrics
            result.append({'Keyword text': criterion.keyword.text,
                           'match type': criterion.keyword.match_type,
                           'ID': criterion.criterion_id,
                           'ad_group': ad_group.name,
                           'group_ID': ad_group.id,
                           'campaign': campaign.name,
                           'campaign_ID': campaign.id,
                           'impressions': metrics.impressions,
                           'clicks': metrics.clicks,
                           'cost_in_micros': metrics.cost_micros})
#             print(
#                 f'Keyword text "{criterion.keyword.text}" with '
#                 f'match type "{criterion.keyword.match_type.name}" '
#                 f"and ID {criterion.criterion_id} in "
#                 f'ad group "{ad_group.name}" '
#                 f'with ID "{ad_group.id}" '
#                 f'in campaign "{campaign.name}" '
#                 f"with ID {campaign.id} "
#                 f"had {metrics.impressions} impression(s), "
#                 f"{metrics.clicks} click(s), and "
#                 f"{metrics.cost_micros} cost (in micros) during "
#                 "the last 7 days."
#             )
                           
    return result

In [16]:
report = basic_report(client, customer_id='2200886572')

In [17]:
pd.DataFrame(report)

,Keyword text,match type,ID,ad_group,group_ID,campaign,campaign_ID,impressions,clicks,cost_in_micros
0,маркет плейс,4,334480537395,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
1,продажи на маркетплейсах,4,792336893039,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
2,маркетплейс обучение,4,878876771959,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
3,работа с маркетплейсами обучение,4,878876772959,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
4,торговля на маркетплейсах обучение,4,878876773199,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
5,аналитика wildberries,4,890376457331,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
6,обучение работы с wildberries,4,891749857913,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
7,wildberries обучение,4,902566131763,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
8,аналитика продаж на маркетплейсах,4,903990930713,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
9,менеджер по работе с маркетплейсами обучение,4,903990931873,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
